In [1]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

In [6]:
// ---------------------------------------------------------------------------
// 05-DFA-2-RegExp (Zelle 1)
// Grundlegende Typdefinitionen für deterministische endliche Automaten (DFA)
// ---------------------------------------------------------------------------

// Ein einzelnes Zeichen des Alphabets
type Char = string;

// Rekursiver Typ für reguläre Ausdrücke
// Entspricht: RegExp = int | Char | tuple[RegExp, ...]
type RegExp = number | Char | [RegExp, ...RegExp[]];

// Zustände werden als Zahlen dargestellt
type State = number;

// Übergangsrelation: (State, Char) -> State
type TransRel = Map<[State, Char], State>;

// Definition eines deterministischen endlichen Automaten (DFA)
interface DFA {
  Q: Set<State>;       // Zustände
  Sigma: Set<Char>;    // Alphabet
  delta: TransRel;     // Übergangsrelation
  q0: State;           // Startzustand
  F: Set<State>;       // akzeptierende Zustände
}

# Converting a Deterministic <span style="font-variant:small-caps;">Fsm</span> into a Regular Expression

The function `regexp_sum` takes a set $S = \{ r_1, \cdots, r_n \}$ of regular expressions
as its argument.  It returns the regular expression 
$$ r_1 + \cdots + r_n. $$
The regular expression will be represented as a nested tuple that uses the operators `+` (for alternatives), `&` (for concatenations), and `*` (for repetitions).

Instead of a set, in the recursive invocation the function is called with a  list.  This compilicates the type annotation a little bit and forces us to annotate the input agrument `S` with `Collection[RegExp]`, since `Collection` is a super type of both `Set` and `List`.

In [7]:
function regexp_sum(S: Set<RegExp> | RegExp[]): RegExp {
  const list = Array.from(S);
  const n = list.length;

  switch (n) {
    case 0:
      return 0; // leere Sprache
    case 1:
      return list[0];
    default: {
      const [r, ...rest] = list;
      return [r, '+', regexp_sum(rest)] as RegExp;
    }
  }
}

The function `rpq` assumes there is some <span style="font-variant:small-caps;">Fsm</span>
$$ F = \langle \texttt{States}, \Sigma, \delta, \texttt{q0}, \texttt{Accepting} \rangle $$
given and takes five arguments:
- `p1` and `p2` are states of the <span style="font-variant:small-caps;">Fsm</span> $F$,
- $\Sigma$ is the alphabet of the <span style="font-variant:small-caps;">Fsm</span>,
- $\delta$ is the transition function of the <span style="font-variant:small-caps;">Fsm</span> $F$, and
- `Allowed` is a subset of the set `States`.  On recursive calls, `Allowed` is a list of states.

The function `rpq` computes a regular expression that describes those strings that take the 
<span style="font-variant:small-caps;">Fsm</span> $F$ from the state `p1` to state `p2`.
When $F$ switches states from `p1` to `p2` only states in the set `Allowed` may be visited in-between the states `p1` and `p2`.

The function is defined by recursion on the set `Allowed`.  There are two cases
- $\texttt{Allowed} = \{\}$.  
  Define `AllChars`as the set of all characters that when read by $F$ in the state `p1` cause $F$ to enter the state `p2`:
  $$ \texttt{AllChars} = \{ c \in \Sigma \mid \delta(p_1, c) = p_2 \} $$
  Then we need a further case distinction:
  - $p_1 = p_2$: In this case we have:
    $$ \texttt{rpq}(p_1, p_2, \{\}) := \sum\limits_{c\in\texttt{AllChars}} c \quad + \varepsilon$$
    If $\texttt{AllChars} = \{\}$ the sum $\sum\limits_{c\in\texttt{AllChars}} c$ is to be interpreted as the
    regular expression $\emptyset$ that denotes the empty language. 
    
    Otherwise, if $\texttt{AllChars} = \{c_1,\cdots,c_n\}$ we have
    $\sum\limits_{c\in\texttt{AllChars}} c \quad = c_1 + \cdots + c_n$.
  - $p_1 \not= p_2$: In this case we have:
    $$ \texttt{rpq}(p_1, p_2, \{\}) := \sum\limits_{c\in\texttt{AllChars}} c \quad$$
- $\texttt{Allowed} = \{ q \} \cup \texttt{RestAllowed}$.  In this case we recursively define the following variables:
  1. $\texttt{rp1p2} := \texttt{rpq}(p_1, p_2, \Sigma, \delta, \texttt{RestAllowed})$,
  2. $\texttt{rp1q } := \texttt{rpq}(p_1, q, \Sigma, \delta, \texttt{RestAllowed})$,
  3. $\texttt{rqq }\texttt{ } := \texttt{rpq}(q, q, \Sigma, \delta, \texttt{RestAllowed})$,
  4. $\texttt{rqp2 } := \texttt{rpq}(q, p_2, \Sigma, \delta, \texttt{RestAllowed})$.

  Then we can define:
  $$ \texttt{rpq}(p_1, p_2, \texttt{Allowed}) := \texttt{rp1p2} + \texttt{rp1q} \cdot \texttt{rqq}^* \cdot \texttt{rqp2} $$
  This formula can be understood as follows:  If a string $w$ is read in state $p_1$ and reading this string takes the 
  <span style="font-variant:small-caps;">Fsm</span> $F$ from the state $p_1$ to the state $p_2$ while only visiting states from the set 
  `Allowed` in-between, then there are two cases:
  - Reading $w$ does not visit the state $q$ in-between.  Hence the string $w$ can be described by the regular expression
    `rp1p2`.
  - The string $w$ can be written as $w = t u_1 \cdots u_n v$ where:
    - reading $t$ in the state $p_1$ takes the <span style="font-variant:small-caps;">Fsm</span> $F$ into the state $q$,
    - for all $i \in \{1,\cdots,n\}$ reading $v_i$ in the state $q$ takes the <span style="font-variant:small-caps;">Fsm</span> $F$ from $q$ to $q$, and
    - reading $v$ in the state $q$ takes the <span style="font-variant:small-caps;">Fsm</span> $F$ into the state $p_2$.

In [8]:
function rpq(
  p1: State,
  p2: State,
  Sigma: Set<Char>,
  delta: TransRel,
  Allowed: State[] | Set<State>
): RegExp {
  const allowedList = Array.from(Allowed);

  // Fall 1: Allowed ist leer
  if (allowedList.length === 0) {
    const AllChars = new Set<RegExp>(
      Array.from(Sigma).filter((c) => {
        for (const [[q, sym], target] of delta.entries()) {
          if (q === p1 && sym === c && target === p2) return true;
        }
        return false;
      })
    );

    const r = regexp_sum(AllChars);

    if (p1 === p2) {
      // ε + r
      return ['ε', '+', r] as RegExp;
    } else {
      return r;
    }
  }

  // Fall 2: Allowed = {q} ∪ RestAllowed
  const [q, ...RestAllowed] = allowedList;

  const rp1p2 = rpq(p1, p2, Sigma, delta, RestAllowed);
  const rp1q = rpq(p1, q, Sigma, delta, RestAllowed);
  const rqq = rpq(q, q, Sigma, delta, RestAllowed);
  const rqp2 = rpq(q, p2, Sigma, delta, RestAllowed);

  // rpq(p1,p2,Allowed) := rp1p2 + rp1q ⋅ rqq* ⋅ rqp2
  return [rp1p2, '+', [[rp1q, '⋅', [rqq, '*']], '⋅', rqp2]] as RegExp;
}


The function `dfa_2_regexp` takes a deterministic <span style="font-variant:small-caps;">Fsm</span> $F$ and computes a regular expression $r$ that describes the same language as $F$, i.e. we have
$$ L(A) = L(r). $$

In [10]:
function dfa_2_regexp(F: DFA): RegExp {
    const States = F.Q;
    const Sigma = F.Sigma;
    const delta = F.delta;
    const q0 = F.q0;
    const Accepting = F.F;

  const regexSet = new Set<RegExp>();
  for (const p of Accepting) {
    regexSet.add(rpq(q0, p, Sigma, delta, Array.from(States)));
  }

  return regexp_sum(regexSet);
}

The notebook `06-Test-DFA-2-RegExp.ipynb` provides a test for the function `dfa_2_regexp`.